In [1]:
import pandas as pd
import os
import re

In [10]:
#Read files
xls = pd.ExcelFile('BIDSJson.xlsx')

#path variables
BIDS='./BIDS_data/'
NIMS='./NIMS_data/'

#make BIDS Folder
def makefolder(name):
    if not os.path.exists(name):
        os.makedirs(name)
        
makefolder(BIDS)




In [11]:
participants = xls.parse('participants')
protocol = xls.parse('protocol').iloc[:,:4]
protocol = protocol.dropna(axis=0, thresh=3)


NIMS_dir_ref = protocol.NIMS_scan_title.tolist()
print(NIMS_dir_ref)



['BOLD_EPI_29mm_2sec', 'BOLD_EPI_29mm_2sec', 'BOLD_EPI_29mm_2sec', 'BOLD_EPI_29mm_2sec', 'T1w_9mm_BRAVO']


In [15]:
#Create Participants folders in BIDS
participants.participant_id.apply(lambda x: makefolder(BIDS + 'sub-' + str(x)))

0    None
1    None
2    None
3    None
4    None
Name: participant_id, dtype: object

In [19]:



#Get list of files in participant folder
for index, row in participants.iterrows():
    #Get files
    NIMS_dir_files = os.listdir(NIMS + row.nims_title)
    
    #Get rid of headers
    NIMS_dir_files = [re.sub(r'[0-9]{4}_[0-9]{2}_', r'',x) for x in NIMS_dir_files]
    
    #Delete all non-nii.gz files
    NIMS_dir_files = [x for x in NIMS_dir_files if ".nii.gz"  in x]
    
    #Remove .nii.gz
    NIMS_dir_files = [re.sub(r'.nii.gz', r'',x) for x in NIMS_dir_files]

    #Remove files not in NIMS_dir_ref
    NIMS_dir_files = [x for x in NIMS_dir_files if x in NIMS_dir_ref]

    
    for item in set(NIMS_dir_ref):
        if NIMS_dir_files.count(item) == NIMS_dir_ref.count(item):
            print(str(row.participant_id) + ": ++" + item + " file counts match, moving to BIDS_data")
            
        else:
            print(str(row.participant_id) + ": --"+ item + " file counts do not match protocol")


222: ++BOLD_EPI_29mm_2sec file counts match, moving to BIDS_data
222: ++T1w_9mm_BRAVO file counts match, moving to BIDS_data
262: ++BOLD_EPI_29mm_2sec file counts match, moving to BIDS_data
262: ++T1w_9mm_BRAVO file counts match, moving to BIDS_data
154: ++BOLD_EPI_29mm_2sec file counts match, moving to BIDS_data
154: ++T1w_9mm_BRAVO file counts match, moving to BIDS_data
231: ++BOLD_EPI_29mm_2sec file counts match, moving to BIDS_data
231: ++T1w_9mm_BRAVO file counts match, moving to BIDS_data
280: ++BOLD_EPI_29mm_2sec file counts match, moving to BIDS_data
280: ++T1w_9mm_BRAVO file counts match, moving to BIDS_data


In [ ]:
[x for x in ]